# Foundations of Artificial Intelligence and Machine Learning
## A Program by IIIT-H and TalentSprint

The objective of this experiment is to understand Siamese Network.

Tons of data area available on the web (wikipedia, Google, Twitter, YouTube) that could be used to train an ML model.

---



---


One such source is Google Images. You enter a text query and Google Images shows thousands of related images based on the query and text that are present on the web page with the related image.

In this experiment we would crawl images from Google Images and try to use this as data for training.

1. Your task is to search for face images for 'AamairKhan', 'Rimisen', 'Kajol', 'KareenaKapoor','RishiKapoor', 'AmrishPuri', 'AnilKapoor', 'AnupamKher', 'BomanIrani', 'HrithikRoshan', 'KajalAgarwal', 'KatrinaKaif', 'Madhavan', 'MadhuriDixit', 'Umashri', 'Trisha'

2. Refine your search to faces (Google Images -> enter query -> Tools -> Type -> Face). You could also use movies', ads' names as additional query (e.g., "Aamir 3 idiots", "Boman Irani Khosla Ka Ghosla", "Katrina Slice ad" etc.). The results are noisy but they are useful, and moreover, they are avaible in abundance and for free!

    a. Example: https://www.google.co.in/search?client=firefox-b-ab&dcr=0&biw=1366&bih=628&tbs=itp%3Aface&tbm=isch&sa=1&ei=5gbIWtCjN4n2vgSCoqzYBw&q=biswa+kalyan+rath

3. Then use a browser extensions to download all the results into a directory. In this way you, would get around 300-600 images for each class. Overall, you should collect atleast 10000 images.
    
    a. Firefox: https://addons.mozilla.org/en-US/firefox/addon/google-images-downloader/
    
    b. Chrome: https://chrome.google.com/webstore/detail/download-all-images/ifipmflagepipjokmbdecpmjbibjnakm/related?hl=en
    
4. **Without cleaning** use these images as your training data. Test you results on IMFDB test set.



#### Run the Notebook on GPU

#### Authenticating Google drive

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

#### Downloading the data

In [1]:
!wget https://www.dropbox.com/s/0b4txuf4si3659z/One_shot_Face_recognition.zip?dl=1
#!ls

--2018-09-30 22:11:00--  https://www.dropbox.com/s/0b4txuf4si3659z/One_shot_Face_recognition.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.1
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.1|:443... connected.
OpenSSL: error:140770FC:SSL routines:SSL23_GET_SERVER_HELLO:unknown protocol
Unable to establish SSL connection.


#### Unziping the downloaded file

In [4]:
!unzip -qq -o One_shot_Face_recognition.zip

#### Installing Pytorch

In [0]:
!pip3 install torch torchvision

#### Changing the Working directory


In [4]:
%cd One_shot_Face_recognition

/Users/tgupta2/Documents/Training_Courses/AIML/Session10/One_shot_Face_recognition


#### Importing the Required Packages

In [5]:
# Importing pytorch packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
# Importing config.py file
import config as cf
from utils import *
from light_cnn import LightCNN_9Layers #, LightCNN_29Layers, LightCNN_29Layers_v2
#from resnet import resnet18
from siamese_data_loader import *
from contrastive import *   ### implementation of contrastive loss
## Importing python packages
import os
import sys
import time
import datetime
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle

In [6]:
print(cf.data_dir)

data/


#### Loading the data 

In [26]:
img_root = cf.data_dir+'IMFDB_final/'

train_list_file = cf.data_dir+'IMFDB_train_sorted.txt'   #### 5000 images for training
val_list_file = cf.data_dirimg_root = cf.data_dir+'IMFDB_final/'

train_list_file = cf.data_dir+'IMFDB_train_sorted.txt'   #### 5000 images for training
val_list_file = cf.data_dir+'IMFDB_test_sorted.txt'      #### 1095 images for validation


train_image_list = [line.rstrip('\n') for line in open(train_list_file)]
val_image_list = [line.rstrip('\n') for line in open(val_list_file)]

print(len(train_image_list), len(val_image_list))

### Notice a new data loader for siamese networks. This gives the image pairs (image_1, image_2) and a label as input to the siamese networks.
### see siamese_data_loader.py for details

trainloader = torch.utils.data.DataLoader(siamese_data_loader(img_root = img_root, image_list = train_list_file, crop=False,
                                                             resize = True, resize_shape=[128,128]), 
batch_size=32, num_workers=1, shuffle = False, pin_memory=False)

testloader = torch.utils.data.DataLoader(siamese_data_loader(img_root = img_root, image_list = val_list_file, crop=False, mirror=False, 
                                                           resize = True, resize_shape=[128,128]), 
                                           batch_size=10, num_workers=1, shuffle = False, pin_memory=False)


#classes = ['AamairKhan', 'Rimisen', 'Kajol', 'KareenaKapoor','RishiKapoor', 'AmrishPuri', 'AnilKapoor', 'AnupamKher', 'BomanIrani', 'HrithikRoshan', 'KajalAgarwal', 'KatrinaKaif', 'Madhavan', 'MadhuriDixit', 'Umashri', 'Trisha']+'IMFDB_test_sorted.txt'      #### 1095 images for validation


train_image_list = [line.rstrip('\n') for line in open(train_list_file)]
val_image_list = [line.rstrip('\n') for line in open(val_list_file)]

print(len(train_image_list), len(val_image_list))

### Notice a new data loader for siamese networks. This gives the image pairs (image_1, image_2) and a label as input to the siamese networks.
### see siamese_data_loader.py for details

trainloader = torch.utils.data.DataLoader(siamese_data_loader(img_root = img_root, image_list = train_list_file, crop=False,
                                                             resize = True, resize_shape=[128,128]), 
batch_size=20, num_workers=1, shuffle = False, pin_memory=False)

testloader = torch.utils.data.DataLoader(siamese_data_loader(img_root = img_root, image_list = val_list_file, crop=False, mirror=False, 
                                                           resize = True, resize_shape=[128,128]), 
                                           batch_size=5, num_workers=1, shuffle = False, pin_memory=False)


classes = ['AamairKhan', 'Rimisen', 'Kajol', 'KareenaKapoor','RishiKapoor', 'AmrishPuri', 'AnilKapoor', 'AnupamKher', 'BomanIrani', 'HrithikRoshan', 'KajalAgarwal', 'KatrinaKaif', 'Madhavan', 'MadhuriDixit', 'Umashri', 'Trisha']

5000 1095
5000 1095


#### Command to check whether GPU is enabled or not


In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:

#Intilizaing the loss value as high value
best_loss = 99999999

num_classes = 16

In [15]:
from torchvision import models 

In [16]:
feature_net = LightCNN_9Layers()

In [17]:
#feature_net = torch.load(cf.data_dir+'lightCNN_51_checkpoint.pth', map_location='cpu' )

In [30]:
#deep feature net with 9 layers, we will train this
feature_net = LightCNN_9Layers()   ### creates an object of this network architecture
#feature_net = torch.load(cf.data_dir+'lightCNN_51_checkpoint.pth', map_location='cpu' )


#layers_to_remove = ['fc2']
#for layers_ in layers_to_remove:        
#    del(feature_net._modules[layers_])
    
classifier = nn.Sequential(nn.Linear(256, 64), nn.BatchNorm1d(64), nn.ReLU(),
                           nn.Linear(64, 32), nn.BatchNorm1d(32), nn.ReLU(),
                           nn.Linear(32, num_classes))

feature_net.fc2 = nn.Sequential(nn.Linear(256, 16))
feature_net = feature_net.to(device)
classifier =  classifier.to(device)
    


In [31]:
### Intiliazing the loss
criterion = nn.CrossEntropyLoss()
siamese_loss = contrastive_loss()   ### Notice a new loss. contrastive.py shows how to compute contrastive loss.

In [32]:
criterion = criterion.to(device)
siamese_loss = siamese_loss.to(device)

#### Lets train the siamese networks. The objective is images from same class (+ pair, label = 0) should have similar feature and images from different classes (- pair, label = 1) should have different features. Instead of having two physical networks sharing the weights, in implementation we have only one network and first pass image_1 (to get its feature) and then pass image_2 (to get its feature) through the same network. We then compute the contrastive loss on these feature pairs from input image pairs. This saves a lot of memory.

In [33]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    feature_net.train()
    train_loss = 0
    correct = 0
    total = 1
    #the trainloader gives 3 things, 2 images and a label whether they are similar or not
    for batch_idx, (inputs_1, inputs_2, targets) in enumerate(trainloader):
        inputs_1, inputs_2, targets = inputs_1.to(device), inputs_2.to(device), targets.to(device)
        optimizer.zero_grad()
        #inputs_1, inputs_2, targets = inputs_1), Variable(inputs_2), Variable(targets)
        features_1 = feature_net(inputs_1)[1]     ### get feature for image_1
        features_2 = feature_net(inputs_2)[1]      ### get feature for image_2
        
        loss = siamese_loss(features_1, features_2, targets.float())   ### compute the contrastive loss, computes the similarity between the features.
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        #print(1)
        
        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f '% (train_loss/(batch_idx+1)))
        
    train_loss_file.write('%d %.3f %.3f\n' %(epoch, train_loss/len(trainloader), 100.*correct/total))
        #print(1)



#### Function to test



In [34]:
def test(epoch):
    global best_loss
    feature_net.eval()
    test_loss = 0
    correct = 0
    total = 1
    for batch_idx, (inputs_1, inputs_2, targets) in enumerate(testloader):
        inputs_1, inputs_2, targets = inputs_1.to(device), inputs_2.to(device), targets.to(device)
        optimizer.zero_grad()
        #inputs_1, inputs_2, targets = Variable(inputs_1), Variable(inputs_2), Variable(targets)
        features_1 = feature_net(inputs_1)[1]     ### get feature for image_1
        features_2 = feature_net(inputs_2)[1]      ### get feature for image_2      
        
        loss = siamese_loss(features_1, features_2, targets.float())
        test_loss += loss.item()
        
        progress_bar(batch_idx, len(testloader), 'Loss: %.3f '
                         % (test_loss/(batch_idx+1)))
        
    val_loss_file.write('%d %.3f %.3f\n' %(epoch,  test_loss/len(testloader), 100.*correct/total))

    # Save checkpoint.
    losss = test_loss/len(testloader)
    if  losss < best_loss:   ### save model with the best loss so far
        print('Saving..') 
        state = {
            'net': feature_net
        }
        if not os.path.isdir(cf.data_dir+'checkpoint'):
            os.mkdir(cf.data_dir+'checkpoint')
        torch.save(state, cf.data_dir+'checkpoint/siamese_ckpt.t7')
        best_loss = losss
    
    return test_loss/len(testloader)

#### Creating the files to store train and validation data loss values

In [35]:
experiment = 'siamese_IMFDB/'
train_loss_file = open(cf.data_dir+experiment+"train_loss.txt", "w")
val_loss_file = open(cf.data_dir+experiment+"val_loss.txt", "w")

In [ ]:
feature_net = feature_net.to(device)
optimizer = optim.Adam(feature_net.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True)   #### dynamic LR scheduler
for epoch in range(0, 10):
    train(epoch)
    test_loss = test(epoch)
    scheduler.step(test_loss)
    print("Test Loss: ", test_loss)
train_loss_file.close()
val_loss_file.close()


Epoch: 0


In [0]:
### After training we load the model that performed the best on validation data (avoid picking overfitted model)
### we will use the base pre-trained network for feature extraction only. This feature is used to train an MLP classifier.

feature_net = torch.load(cf.data_dir+'checkpoint/siamese_ckpt.t7',map_location='cpu' )['net'].eval()

#### Lets see how well does the siamese detect an imposter. We check whether image_2 is same individual as image_1 or an imposter. We do this by computing dissimilarity score between features.

In [0]:
testloader = torch.utils.data.DataLoader(siamese_data_loader(img_root = img_root, image_list = val_list_file, crop=False, mirror=False, 
                                                           resize = True, resize_shape=[128,128]), 
                                           batch_size=1, num_workers=1, shuffle = False, pin_memory=False)

lab = ['same', 'imposter']
with torch.no_grad():
    for batch_idx, (inputs_1, inputs_2, targets) in enumerate(testloader):
        if batch_idx%10 == 0 or int(targets)==0:      ### show every tenth image or if its the same individual

            inputs_1, inputs_2, targets = inputs_1.to(device), inputs_2.to(device), targets.to(device)
            optimizer.zero_grad()
            #inputs_1, inputs_2, targets = inputs_1), Variable(inputs_2), Variable(targets)
            features_1 = feature_net(inputs_1)[1]     ### get feature for image_1
            features_2 = feature_net(inputs_2)[1]      ### get feature for image_2

            dissimilarity = torch.nn.functional.cosine_similarity(features_1, features_2).item()
            img = np.concatenate((inputs_1.data.cpu().numpy()[0][0], inputs_2.data.cpu().numpy()[0][0]), axis = 1)
            plt.imshow(img, cmap='gray')
            plt.text(100,20,str(dissimilarity), fontsize=24, color='r')     ### similarity score
            plt.text(100,40,lab[int(targets.data[0])], fontsize=24, color='r')   ### ground truth
            plt.show()


### Now we use this network for feature extraction and train an MLP classifier. Feature_net is not updated/train/tweak after this. We only train the MLP classifier.

---



In [0]:
#from google.colab import files 
#uploaded = files.upload()

import collections
import os.path as osp

import numpy as np
import PIL.Image
import cv2
import scipy.io
import torch
from torch.utils import data
from random import shuffle
import os.path

import os
import sys
import copy
import time
import math
import matplotlib.pyplot as plt

class custom_data_loader(data.Dataset):    ### custom data loader
    
    def __init__(self, img_root, image_list, mirror = True, crop = True, crop_shape = [256, 256],
                 resize=False, resize_shape=[128,128], split = 'train', classes = 'IMFDB'):

        self.img_root = img_root
        self.split = split
        
        self.image_list = [line.rstrip('\n') for line in open(image_list)]
        
        self.classes = None
        if classes == 'IIC':
            self.classes = ['baba_ramdev', 'biswa',  'dhinchak_pooja',  'khali',  'priya_prakash']
        elif classes == 'IMFDB':
            self.classes = ['AamairKhan', 'Rimisen', 'Kajol', 'KareenaKapoor','RishiKapoor', 'AmrishPuri', 'AnilKapoor', 'AnupamKher', 'BomanIrani', 'HrithikRoshan', 'KajalAgarwal', 'KatrinaKaif', 'Madhavan', 'MadhuriDixit', 'Umashri', 'Trisha']

        self.mirror = mirror
        self.crop = crop
        self.crop_shape = crop_shape
        
        self.resize = resize
        self.resize_shape = resize_shape

        #self.mean_bgr = np.array([123.68, 116.779, 103.939])   ### mean BGR of ImageNet
        self.mean_bgr = np.array([0])   ### lightnet does not perform mean subtraction of input

        self.files = collections.defaultdict(list)
        for f in self.image_list:
            self.files[self.split].append({'img': img_root+f, 'lbl': 0})
        
    def __len__(self):
        return len(self.files[self.split])

    def __getitem__(self, index):
        
        image_file_name = self.img_root + self.image_list[index]
        
        image = None
        if os.path.isfile(image_file_name):
            image = cv2.imread(image_file_name, 0)  ### 0 indicates load image as grayscale image
        if image is None:
            print('creating a default image')
            image = np.zeros((self.resize_shape[1], self.resize_shape[0]))
            if self.resize :
                image = cv2.resize(image, (self.resize_shape[1], self.resize_shape[0]))   ### resize_shape is in [rows, cols]
            
            image = image.reshape(image.shape[0], image.shape[1], 1)
            
            if self.mirror:
                flip = np.random.choice(2)*2-1    ### randomly flip the image
                image = image[:, ::flip, :]

            if self.crop:
                image = self.get_random_crop(image, self.crop_shape)

            label = [self.classes.index(i) for i in self.classes if self.image_list[index].split('/')[0] == i][0]
        
            l = torch.IntTensor(1, 1)
            l = label

            return self.transform_image(image), l
            
        else :    
            if self.resize :
                print('image found resizing')
                image = cv2.resize(image, (self.resize_shape[1], self.resize_shape[0]))   ### resize_shape is in [rows, cols]
            
                image = image.reshape(image.shape[0], image.shape[1], 1)
            
            if self.mirror:
                flip = np.random.choice(2)*2-1    ### randomly flip the image
                image = image[:, ::flip, :]

            if self.crop:
                image = self.get_random_crop(image, self.crop_shape)

            label = [self.classes.index(i) for i in self.classes if self.image_list[index].split('/')[0] == i][0]
        
            l = torch.IntTensor(1, 1)
            l = label

            return self.transform_image(image), l

    def transform_image(self, image):
        image = image.astype(np.float64)
        image -= self.mean_bgr
        image = image.transpose(2, 0, 1)
        image = torch.from_numpy(image.copy()).float()   ### convert from numpy array to torch tensor

        return image


    def get_random_crop(self, im, crop_shape):
        """
        crop shape is of the format: [rows cols]
        """
        r_offset = 0
        c_offset = 0
        
        if im.shape[0] == crop_shape[0]:
            r_offset = 0
        else:
            r_offset = np.random.randint(0, im.shape[0] - crop_shape[0] + 1)
            
        if im.shape[1] == crop_shape[1]:
            c_offset = 0
        else:
            c_offset = np.random.randint(0, im.shape[1] - crop_shape[1] + 1)

        crop_im = im[r_offset: r_offset + crop_shape[0], c_offset: c_offset + crop_shape[1], :]

        return crop_im

In [0]:
#from data_loader import custom_data_loader

train_list_file = cf.data_dir+'IMFDB_train.txt'   #### 5000 images for training
val_list_file = cf.data_dir+'IMFDB_test.txt'      #### 1095 images for validation


train_image_list = [line.rstrip('\n') for line in open(train_list_file)]
val_image_list = [line.rstrip('\n') for line in open(val_list_file)]

print(len(train_image_list), len(val_image_list))

trainloader = torch.utils.data.DataLoader(custom_data_loader(img_root = img_root, image_list = train_list_file, crop=False,
                                                             resize = True, resize_shape=[128,128]), 
                                          batch_size=32, num_workers=16, shuffle = True, pin_memory=False)

testloader = torch.utils.data.DataLoader(custom_data_loader(img_root = img_root, image_list = val_list_file, crop=False, mirror=False, 
                                                           resize = True, resize_shape=[128,128]), 
                                         batch_size=10, num_workers=5, shuffle = False, pin_memory=False)

In [0]:
def train_classifier(epoch):
    print('\nEpoch: %d' % epoch)
    classifier.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        #inputs, targets = Variable(inputs), Variable(targets)
        features = feature_net(inputs)[1]      
        
        
        outputs = classifier(features)
        size_ = outputs.size()
        outputs_ = outputs.view(size_[0], num_classes)
        loss = criterion(outputs_, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        _, predicted = torch.max(outputs_.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        
        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
    train_loss_file.write('%d %.3f %.3f\n' %(epoch, train_loss/len(trainloader), 100.*correct/total))
    


In [0]:
def test_classifier(epoch):
    global best_acc
    classifier.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        #if device:
        inputs, targets = inputs.to(device), targets.to(device)
        #inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        features = feature_net.get_features(inputs).detach()
        
        outputs = classifier(features)
        size_ = outputs.size()
        outputs_ = outputs.view(size_[0], num_classes)
        loss = criterion(outputs_, targets)

        test_loss += loss.item()
        _, predicted = torch.max(outputs_.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        
        progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
    val_loss_file.write('%d %.3f %.3f\n' %(epoch,  test_loss/len(testloader), 100.*correct/total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': classifier,
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir(cf.data_dir+'checkpoint'):
            os.mkdir(cf.data_dir+'checkpoint')
        torch.save(state, cf.data_dir+'checkpoint/checkpoint_ckpt.t7')
        best_acc = acc
    
    return test_loss/len(testloader)

In [0]:
best_acc = 0
experiment = 'siamese_IMFDB'
train_loss_file = open(cf.data_dir+experiment+"train_loss.txt", "rb", 0)
val_loss_file = open(cf.data_dir+experiment+"val_loss.txt", "rb", 0)

In [0]:
optimizer = optim.Adam(classifier.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True)   #### dynamic LR scheduler
for epoch in range(0, 30):
    train_classifier(epoch)
    test_loss = test_classifier(epoch)
    scheduler.step(test_loss)
    
train_loss_file.close()
val_loss_file.close()

In [0]:
Copy of def eval():
    feature_net.eval()
    classifier.eval()
    
    testloader = torch.utils.data.DataLoader(custom_data_loader(img_root = img_root, image_list = val_list_file, crop=False, mirror=False, 
                                                           resize = True, resize_shape=[128,128]), 
                                           batch_size=1, num_workers=1, shuffle = False, pin_memory=False)
    correct = 0
    total = 0
    conf_mat = np.zeros((num_classes, num_classes))
    total_ = 1e-12+np.zeros((num_classes))
    wrong_predictions = []
    for batch_idx, (inputs, targets) in enumerate(testloader):
        #if use_cuda:
        inputs, targets = inputs.to(device), targets.to(device)
        #inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        features = feature_net.features(inputs).detach()
        features = features.view(1,-1)
        #print(features.size())        
        outputs = classifier(features)
        size_ = outputs.size()
        outputs_ = outputs.view(size_[0], num_classes)
        _, predicted = torch.max(outputs_.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        prediction = predicted.cpu().numpy()[0]
        targets = targets.data.cpu().numpy()[0]
        total_[targets] +=1
        conf_mat[predicted, targets] +=1
        
        if prediction != targets:
            wrong_predictions += [[inputs, prediction, targets]]
        
    for k in range(num_classes):
        conf_mat[:,k] /= total_[k]
    return conf_mat, 100.*correct/total, wrong_predictions
    

In [0]:
classifier = nn.Sequential(nn.Linear(8192, 64), nn.BatchNorm1d(64), nn.ReLU(),
                           nn.Linear(64, 32), nn.BatchNorm1d(32), nn.ReLU(),
                           nn.Linear(32, num_classes))
classifier = classifier.to(device)

In [0]:
conf, acc, wrong_predictions = eval()
print(acc)

In [0]:
plt.imshow(conf, cmap='jet', vmin=0, vmax = 1)
plt.show()

In [0]:
for w in wrong_predictions[::10]:
    print(classes[w[2]], 'confused with', classes[w[1]])
    plt.imshow(w[0][0][0].data.cpu().numpy(), cmap='gray')
    plt.show()